In [ ]:
import sys
import delivery_driver as dd
sys.path.append("../../deliveryrobot")
from utilities.utilities import *

In [ ]:
robot = dd.DeliveryRobot()

In [ ]:
robot.take_picture(cal_dir)

In [ ]:
from sensors.camera.apriltagsensor import *
from sensors.calibration.camera_calibration import *

In [ ]:
calibrate_fisheye_checkerboard(cal_dir)